This is an example notebook to scrape fbref for all matches from a given season in a given league

We loosely follow this article:
https://levelup.gitconnected.com/how-to-get-football-data-with-code-examples-for-python-and-r-8d3f29938d93

In [27]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [28]:
pd.set_option('display.max_rows', 1000)


url = 'https://fbref.com/en/comps/9/2022-2023/schedule/2022-2023-Premier-League-Scores-and-Fixtures'

prem_list = []

prem_list.append(pd.read_html(url,index_col=False,flavor='lxml')[0])
prem_list = pd.concat(prem_list, axis=0, ignore_index=True)


In [29]:
prem_df = prem_list[prem_list['Wk'].notna()]

prem_df = prem_df.rename(columns={'xG':'xGHome'
                   ,'xG.1':'xGAway'})


prem_df['HomeScore'] = prem_df['Score'].str[0]
prem_df['AwayScore'] = prem_df['Score'].str[2]

prem_df = prem_df.drop(['Match Report','Notes'],axis=1)

#convert to date

prem_df['Date'] = pd.to_datetime(prem_df['Date'])

prem_df.sort_values(by='Date', inplace=True)

prem_df = prem_df[['Wk','Day','Date','Time','Home','HomeScore','xGHome','AwayScore','xGAway','Away','Attendance','Venue','Referee']]

In [38]:
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')
table = soup.find('table')

match_reports = []
for tr in table.findAll("tr"):
    trs = tr.findAll("td")
    for each in trs:
        try:
            match_report = each.find('a', text=re.compile('Match Report'))['href']
            match_reports.append('https://fbref.com' + match_report)
        except:
            pass

prem_df['Match Report'] = match_reports

C:\Users\Tejas Srivatsav\AppData\Local\Temp\ipykernel_36852\3418015723.py:10: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  match_report = each.find('a', text=re.compile('Match Report'))['href']


In [39]:
prem_df.head(10)

,Wk,Day,Date,Time,Home,HomeScore,xGHome,AwayScore,xGAway,Away,Attendance,Venue,Referee,Match Report
0,1.0,Fri,2022-08-05,20:00,Crystal Palace,0,1.2,2,1.0,Arsenal,25286.0,Selhurst Park,Anthony Taylor,https://fbref.com/en/matches/e62f6e78/Crystal-...
1,1.0,Sat,2022-08-06,12:30,Fulham,2,1.2,2,1.2,Liverpool,22207.0,Craven Cottage,Andy Madley,https://fbref.com/en/matches/6713c1dc/Fulham-L...
2,1.0,Sat,2022-08-06,15:00,Tottenham,4,1.5,1,0.5,Southampton,61732.0,Tottenham Hotspur Stadium,Andre Marriner,https://fbref.com/en/matches/09d8a999/Tottenha...
3,1.0,Sat,2022-08-06,15:00,Newcastle Utd,2,1.7,0,0.3,Nott'ham Forest,52245.0,St James' Park,Simon Hooper,https://fbref.com/en/matches/1ac96eb4/Newcastl...
4,1.0,Sat,2022-08-06,15:00,Leeds United,2,0.8,1,1.3,Wolves,36347.0,Elland Road,Robert Jones,https://fbref.com/en/matches/82702941/Leeds-Un...
5,1.0,Sat,2022-08-06,15:00,Bournemouth,2,0.6,0,0.7,Aston Villa,11013.0,Vitality Stadium,Peter Bankes,https://fbref.com/en/matches/877e3193/Bournemo...
6,1.0,Sat,2022-08-06,17:30,Everton,0,0.7,1,1.5,Chelsea,39254.0,Goodison Park,Craig Pawson,https://fbref.com/en/matches/3a917cee/Everton-...
7,1.0,Sun,2022-08-07,14:00,Leicester City,2,0.6,2,0.8,Brentford,31794.0,King Power Stadium,Jarred Gillett,https://fbref.com/en/matches/3249ba27/Leiceste...
8,1.0,Sun,2022-08-07,14:00,Manchester Utd,1,1.4,2,1.5,Brighton,73711.0,Old Trafford,Paul Tierney,https://fbref.com/en/matches/8251694e/Manchest...
9,1.0,Sun,2022-08-07,16:30,West Ham,0,0.5,2,2.2,Manchester City,62443.0,London Stadium,Michael Oliver,https://fbref.com/en/matches/ece62baf/West-Ham...


In [40]:
prem_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 380 entries, 0 to 429
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Wk            380 non-null    float64       
 1   Day           380 non-null    object        
 2   Date          380 non-null    datetime64[ns]
 3   Time          380 non-null    object        
 4   Home          380 non-null    object        
 5   HomeScore     380 non-null    object        
 6   xGHome        380 non-null    float64       
 7   AwayScore     380 non-null    object        
 8   xGAway        380 non-null    float64       
 9   Away          380 non-null    object        
 10  Attendance    379 non-null    float64       
 11  Venue         380 non-null    object        
 12  Referee       380 non-null    object        
 13  Match Report  380 non-null    object        
dtypes: datetime64[ns](1), float64(4), object(9)
memory usage: 44.5+ KB


In [41]:
prem_df.to_csv('Datasets/prem_match_data_example.csv', index_label = False)